# Regression Models with Keras

### Instrucctions

A. Build a baseline model (5 marks)

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the
train_test_split
helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

Submit your Jupyter Notebook with your code and comments.


B. Normalize the data (5 marks)

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?


D. Increase the number of hidden layers (5 marks)

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?

### Installing and importing

In [1]:
# installing scikit-learn if not installed
# !pip show scikit-learn

In [2]:
# importing all the libraries needed
import pandas as pd
import numpy as np
import sklearn

## Let's download the data and read it into a pandas dataframe.

In [4]:
concrete_data = pd.read_csv('/content/drive/MyDrive/IBM_AI_Engineering_Professional_Certificate/Course 2 Introduction to Deep Learning & Neural Networks with Keras/Course_Project/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [7]:
concrete_data.shape

(1030, 9)

## 1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_split helper function from Scikit-learn.

***Split data into training and test sets***

In [8]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
def split_train_test(concrete_data):
  train_data, test_data = train_test_split(concrete_data, test_size=0.3, train_size=0.7, random_state=42, shuffle=True, stratify=None)
  # print(f"Training dataset is 70% = {train_data.shape}")
  # print(f"Training dataset is 30% = {test_data.shape}")
  return train_data, test_data

***Split data into predictors and target***

The target variable in this problem is the concrete sample strength and the predictors are all the other columns except age.

In [9]:
concrete_data_columns = concrete_data.columns
def split_pred_target(train_data, test_data):
  predictors_train  = train_data[concrete_data_columns[(concrete_data_columns != 'Strength') & (concrete_data_columns != 'Age')]] # all columns except Strength and Age
  target_train = train_data['Strength'] # Strength column
  # predictors_train.head()
  # target_train.head()
  predictors_test = test_data[concrete_data_columns[(concrete_data_columns != 'Strength') & (concrete_data_columns != 'Age')]] # all columns except Strength and Age
  target_test = test_data['Strength'] # Strength column
  # predictors_test.head()
  # target_test.head()
  return (predictors_train, target_train, predictors_test, target_test )


***Normalize the data***

In [10]:
def normalize(x):
  x_norm = (x - x.mean()) / x.std()
  return x_norm


def normalize_data(predictors_train, target_train, predictors_test, target_test):
  predictors_train_norm = normalize(predictors_train)
  # predictors_train_norm.head()
  target_train_norm = normalize(target_train)
  # target_train_norm.head()
  predictors_test_norm = normalize(predictors_test)
  # predictors_test_norm.head()
  target_test_norm = normalize(target_test)
  # target_test_norm.head()
  return (predictors_train_norm, target_train_norm, predictors_test_norm, target_test_norm)

In [11]:
n_cols = 7
# print(n_cols)

## 2. Train the model on the training data using 50 epochs.
## 3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

***Build a Neural Network***

Import Keras and important modules

In [12]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

Define regression model

In [13]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))  # since it has seven(7) predictors
    model.add(Dense(10, activation='relu', ))  # since it has seven(7) predictors
    model.add(Dense(10, activation='relu', ))  # since it has seven(7) predictors
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [14]:
# build the model
model = regression_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                80        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 311 (1.21 KB)
Trainable params: 311 (1.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [15]:
from sklearn.metrics import mean_squared_error
mse = []

for i in range(50):
  train_data, test_data = split_train_test(concrete_data)

  predictors_train, target_train, predictors_test, target_test = split_pred_target(train_data, test_data)

  predictors_train_norm, target_train_norm, predictors_test_norm, target_test_norm = normalize_data(predictors_train, target_train, predictors_test, target_test)

  # fit the model
  model.fit(predictors_train_norm, target_train_norm, validation_split=0.3, epochs=50, verbose=2)
  prediction_test = model.predict(predictors_test_norm, batch_size=None, verbose="auto", steps=None, callbacks=None)

  mse.append(mean_squared_error(target_test_norm, prediction_test))

Streaming output truncated to the last 5000 lines.
16/16 - 0s - loss: 0.4852 - val_loss: 0.5083 - 113ms/epoch - 7ms/step
Epoch 27/50
16/16 - 0s - loss: 0.4845 - val_loss: 0.5074 - 106ms/epoch - 7ms/step
Epoch 28/50
16/16 - 0s - loss: 0.4758 - val_loss: 0.5031 - 100ms/epoch - 6ms/step
Epoch 29/50
16/16 - 0s - loss: 0.4735 - val_loss: 0.5021 - 103ms/epoch - 6ms/step
Epoch 30/50
16/16 - 0s - loss: 0.4697 - val_loss: 0.5006 - 99ms/epoch - 6ms/step
Epoch 31/50
16/16 - 0s - loss: 0.4666 - val_loss: 0.5029 - 107ms/epoch - 7ms/step
Epoch 32/50
16/16 - 0s - loss: 0.4634 - val_loss: 0.5010 - 105ms/epoch - 7ms/step
Epoch 33/50
16/16 - 0s - loss: 0.4604 - val_loss: 0.5008 - 106ms/epoch - 7ms/step
Epoch 34/50
16/16 - 0s - loss: 0.4575 - val_loss: 0.4975 - 113ms/epoch - 7ms/step
Epoch 35/50
16/16 - 0s - loss: 0.4539 - val_loss: 0.4975 - 108ms/epoch - 7ms/step
Epoch 36/50
16/16 - 0s - loss: 0.4514 - val_loss: 0.5002 - 114ms/epoch - 7ms/step
Epoch 37/50
16/16 - 0s - loss: 0.4493 - val_loss: 0.4944 - 1



---



---



In [16]:
print(mse)  # list of 50 mean squared errors
len(mse)

[0.5805726258746996, 0.6083153431612662, 0.6084096535676949, 0.6191818512539021, 0.6405976527518258, 0.6473605425830622, 0.6478854978719959, 0.6622773371978617, 0.6594060170096415, 0.6627476424926723, 0.6670149299125053, 0.6751112124125379, 0.6757433671787824, 0.669489082320245, 0.6768865721090632, 0.6665061138976985, 0.6778219735914952, 0.6772344202041806, 0.6844432328509741, 0.6831437787563408, 0.6947406382154933, 0.6873884728217965, 0.6938952287607534, 0.6902027324470589, 0.6894495644982513, 0.6916778943358871, 0.7012537179051181, 0.7047217522355574, 0.7111502102212242, 0.7091035161858202, 0.7151068138539126, 0.722156307316333, 0.7305454162534116, 0.7291604406543623, 0.7429295587676116, 0.7414796665700011, 0.7475693511845102, 0.7524764945654139, 0.7619621395950337, 0.7546754916965703, 0.7647527491158678, 0.7514542099493049, 0.7585355604086257, 0.7666679127043395, 0.7670890376896593, 0.7631308647827202, 0.7787031914566768, 0.7809938387679582, 0.7752430445767843, 0.7707466185662906]


50

## 5. Report the mean and the standard deviation of the mean squared errors.

In [17]:
#  The  mean and the standard deviation of the mean squared errors
mean = sum(mse) / len(mse)
variance = sum([((x - mean) ** 2) for x in mse]) / len(mse)
std = variance ** 0.5

print(f'mean and the standard deviation of the mean squared errors is {mean:.5f} and {std:.5f} respectively.')


mean and the standard deviation of the mean squared errors is 0.70278 and 0.04947 respectively.


**If we compare the mean of the mean squared errors to that of part B they are bigger.**